In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data = pd.read_csv("/kaggle/input/google-play-store-cleaned/Google_data_cleaned.csv")

In [ ]:
data

In [ ]:
data.rename(columns={'Unnamed: 0':'ID'},inplace = True)

In [ ]:
data

In [ ]:
data.info()

In [ ]:
data.shape

In [ ]:
data["ID"].dtypes
#ID변수 설정 1

In [ ]:
#2
data["ID"].isnull().sum()

In [ ]:
n = len (pd.unique(data["ID"]))
print(n)
#3

In [ ]:
mask = pd.to_numeric(data["ID"],errors='coerce').isna()
a = mask.sum()
print(a)
#4

In [ ]:
data["rating"].dtype
#타겟변수 rating

In [ ]:
data["rating"].describe()

In [ ]:
pd.crosstab(data['rating'], columns = 'count') 
#도수분포표(타겟변수) 생성(개수 기준)

In [ ]:
pd.crosstab(data['rating'], columns = 'ratio', normalize=True)
#도수분포표(타겟변수)생성 (비율 기준)

In [ ]:
data["rating"].isnull().sum()

In [ ]:
data["rating"].isnull().mean()

In [ ]:
data["rating"].value_counts(dropna=False)

In [ ]:
rating_null = data[data["rating"].isna()].index

In [ ]:
current_null = data[data["current_ver"].isna()].index

In [ ]:
android_null = data[data["android_ver"].isna()].index

In [ ]:
data.drop(index=rating_null, inplace = True)
data.shape
#타겟변수 결측값이 1462개라 1462개를 제거

In [ ]:
size_null = data[data["size(kb)"].isna()].index

In [ ]:
data.drop(index = size_null, inplace = True)

In [ ]:
data = data.dropna(axis = 0)

In [ ]:
value = data.isnull().sum()
#data 변수들 결측값 확인
print(value)

#우리 총 데이터 수가 9658
#결측값이 50%이상이면 변수 삭제
#우리는 삭제할 필요 x

In [ ]:
data["rating"].describe()
#중위수 4.3(아주좋음)

In [ ]:
data.loc[data['rating']>= 4.3, "rating_A"] = 1
data.loc[data['rating']<4.3, "rating_A"] = 0

In [ ]:
#기타 변수 데이터 처리
cols = ["reviews","installs","size(kb)"]
pd.options.display.float_format = '{:.2f}'.format
data[cols].describe()
#구간변수를 cols에 저장
#구간변수의 요약통계 구하기
#교수님: month, year은 구간변수 x

In [ ]:
pd.crosstab(data['reviews'], columns = 'count')
#도수분표포(구간변수) 1

In [ ]:
pd.crosstab(data['installs'], columns = 'count')
#도수분표포(구간변수) 2

In [ ]:
pd.crosstab(data['size(kb)'], columns = 'count')
#도수분표포(구간변수) 3

In [ ]:
data[cols].skew()

In [ ]:
data[cols].kurtosis()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

sns.histplot(ax=axes[0], data=data, x="reviews", kde = True, bins = 20)
sns.histplot(ax=axes[1], data=data, x="installs", kde = True, bins = 20)
sns.histplot(ax=axes[2], data=data, x="size(kb)", kde = True, bins = 20)
plt.show()
#히스토그램

In [ ]:
sns.set_style('whitegrid')

fig, axes = plt.subplots(1,3,figsize=(15,4))

sns.boxplot(ax=axes[0], x="reviews", data = data)
sns.boxplot(ax=axes[1], x="installs", data = data)
sns.boxplot(ax=axes[2], x="size(kb)", data = data)


In [ ]:
#이상값 제거
Q1 = data[['reviews','installs','size(kb)']].quantile(0.25)
Q3 = data[['reviews','installs','size(kb)']].quantile(0.75)
IQR = Q3 - Q1
print(IQR)


In [ ]:
Upper = Q3+3.0*IQR
print(Upper)

In [ ]:
c1 = data['installs'] <= 3970000
c2 = data['size(kb)'] <= 123982.9
c3 = data['reviews'] <= 102912

data2 = data[c1&c2&c3]
data2.shape

In [ ]:
fig, axes = plt.subplots(1,3,figsize=(15,4))

sns.boxplot(ax=axes[0], x="reviews", data = data2)
sns.boxplot(ax=axes[1], x="installs", data = data2)
sns.boxplot(ax=axes[2], x="size(kb)", data = data2)

In [ ]:
cols = ['reviews','installs','size(kb)']
round(data2[cols].corr(),2)
#상관관계 검토
#변수제거 필요 x

In [ ]:
import seaborn as sns

corr = data2[cols].corr()  
annot_kws = {"ha": 'center',"va": 'top'}
sns.heatmap(data=corr, annot=True, annot_kws=annot_kws, cmap="YlGnBu");

In [ ]:
# t-검정
from scipy import stats

data_1 = data2[data2['rating'] < 4.3]['reviews']
data_2 = data2[data2['rating'] > 4.3]['reviews']

stats.ttest_ind(data_1, data_2)

In [ ]:
data_1 = data2[data2['rating'] < 4.3]['installs']
data_2 = data2[data2['rating'] > 4.3]['installs']

stats.ttest_ind(data_1, data_2)

In [ ]:
data_1 = data2[data2['rating'] < 4.3]['size(kb)']
data_2 = data2[data2['rating'] > 4.3]['size(kb)']

stats.ttest_ind(data_1, data_2)

In [ ]:
pip install -U imbalanced-learn

In [ ]:
!pip install graphviz

In [ ]:
data2

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

data2['app_encoded'] = OrdinalEncoder().fit_transform(data2['app'].values.reshape(-1,1))


In [ ]:
data2['category_encoded'] = OrdinalEncoder().fit_transform(data2['category'].values.reshape(-1,1))
data2['genres_encoded'] = OrdinalEncoder().fit_transform(data2['genres'].values.reshape(-1,1))
data2['content_encoded'] = OrdinalEncoder().fit_transform(data2['content_rating'].values.reshape(-1,1))

In [ ]:
data2['android_encoded'] = OrdinalEncoder().fit_transform(data2['android_ver'].values.reshape(-1,1))
data2['current_encoded'] = OrdinalEncoder().fit_transform(data2['current_ver'].values.reshape(-1,1))

In [ ]:
data2
#범주형 변수들 값을 숫자형을 변환 시킴.

In [ ]:
cols2 = ['app_encoded','category_encoded','content_encoded','genres_encoded','android_encoded','current_encoded']

In [ ]:
df1 = pd.get_dummies(data2, columns=cols2)
df1 = df1.drop(['app'],axis=1)
df1 = df1.drop(['category'], axis=1)
df1 = df1.drop(['content_rating'], axis = 1)
df1 = df1.drop(['genres'], axis=1)
df1 = df1.drop(['android_ver'], axis=1)
df1 = df1.drop(['current_ver'], axis=1)
df1 = df1.drop(['rating'], axis=1)
#rating은 빼고 이진수로 변경한 rating_A 사용

#정리

In [ ]:
df1.shape

In [ ]:
list(df1.columns)

In [ ]:
cols3 = ['app_encoded_0.0', 'category_encoded_1.0', 'content_encoded_2.0', 'genres_encoded_3.0', 'android_encoded_4.0', 'current_encoded_5.0']
#기준 더미 변수 정해 cols3에 저장

In [ ]:
df1.drop(cols3, axis=1, inplace=True)
df1.shape

In [ ]:
df1.dtypes

In [ ]:
df1 = df1.astype('category')

더미변수가 많은 관계로 전부 category로 변환후 구간변수만 다시 원래 타입으로 변경

In [ ]:
non_dummy = ['reviews','installs','size(kb)']
df1[non_dummy] = df1[non_dummy].astype('int64')

In [ ]:
df1.dtypes

In [ ]:
data10 = df1.drop(['rating_A'],axis=1)
target = df1['rating_A']
from imblearn.under_sampling import RandomUnderSampler
undersample = RandomUnderSampler(sampling_strategy=0.999, random_state=2)
data_under, target_under = undersample.fit_resample(data10, target)
#0.3으로 했을때 샘플 비율이 적다는 오류가 나와 0.9로 했음.

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data_under, target_under, test_size=0.5, random_state=42, stratify=target_under)

print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
lr = LogisticRegression(solver='lbfgs',penalty='none',random_state=0, n_jobs=-1)
model = lr.fit(X_train, y_train)
pred = model.predict(X_test)

print ("Logreg Training set score:{:.5f}".format(model.score(X_train, y_train)))
print ("Logreg Test set score:{:.5f}".format(accuracy_score(y_test, pred)))

#Logistic Regression 모형

In [ ]:
from sklearn.model_selection import GridSearchCV
params = {'solver': ['lbfgs','saga'],'penalty':['none']}

grid_lr = GridSearchCV(lr, param_grid=params, scoring='accuracy', cv=5, n_jobs=-1)
grid_lr.fit(X_train, y_train)

print("GridSearchCV max accuracy:{:.5f}".format(grid_lr.best_score_))
print("GridSearchCV best parameter:", (grid_lr.best_params_))
#그리드 서치

In [ ]:
best_clf = grid_lr.best_estimator_
pred = best_clf.predict(X_test)
print("Accuracy on test set:{:.5f}".format(accuracy_score(y_test, pred)))

In [ ]:
#사이킷런 신경망 모델
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
clf_mlp = MLPClassifier(max_iter = 2000, random_state = 0)

clf_mlp.fit(X_train, y_train)
pred = clf_mlp.predict(X_test)
accuracy = accuracy_score(y_test, pred)

print ("Neural Network Training set score:{:.5f}".format(clf_mlp.score(X_train, y_train)))
print ("Neural Network test set score:{:.5f}".format(accuracy_score(y_test, pred)))

alpha=0.0001이 default
alpha=0.01 - 중간 정도의 weights 의미
alpha=1 stronger weights 의미

In [ ]:
#랜덤트리
from sklearn.ensemble import RandomForestClassifier
clf_rd = RandomForestClassifier(n_estimators = 20, max_depth = 5, random_state = 0)
model = clf_rd.fit(X_train, y_train)

pred = model.predict(X_test)

print("RandomForest test set:{:.5f}".format(accuracy_score(y_test, pred)))
print("RandomForest training set:{:.5f}".format(model.score(X_train, y_train)))

In [ ]:
#오즈비 해석
feature_name = list(data10.columns)
dft = pd.DataFrame(np.round(np.exp(best_clf.coef_),3).transpose(), index = feature_name, columns=['Odds_ratio'])
dft = dft.sort_values(by='Odds_ratio', ascending=False)
dft

In [ ]:
from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier(random_state=0)
model = tree.fit(X_train, y_train)
pred = model.predict(X_test)

print("AUC on training set:{:.5f}".format(model.score(X_train, y_train)))
print("AUC on test set:{:.5f}".format(accuracy_score(y_test, pred)))

In [ ]:
from sklearn.metrics import roc_auc_score
ROC_AUC = roc_auc_score(y_test, best_clf.predict_proba(X_test)[:, 1])
print("ROC AUC on test set:{:.5f}".format(ROC_AUC))